In [4]:
!pip install numpy pandas matplotlib plotly gensim scikit-learn nltk


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import gensim
import nltk
from sklearn.feature_extraction.text import CountVectorizer


In [6]:
import pandas as pd

# Loading the first dataset
data = pd.read_csv(r'C:\Users\b0161166\Documents\LDA_Topic_Modeling_Project\Datasets\MovieReviews\letterboxd-reviews.csv', encoding='ISO-8859-1')

# Loading the second dataset
data2 = pd.read_csv(r'C:\Users\b0161166\Documents\LDA_Topic_Modeling_Project\Datasets\MovieReviews\metacritic-reviews.csv', encoding='ISO-8859-1', on_bad_lines='skip')

# Display the first few rows of both datasets
print(data.head())
print(data2.head())


                                 Movie name Release Year           Rating  \
0                           Aftersun (2022)         2022   â??â??â??â??Â½   
1                              Joker (2019)         2019  â??â??â??â??â??   
2       Puss in Boots: The Last Wish (2022)         2022            â??Â½   
3          The Banshees of Inisherin (2022)         2022  â??â??â??â??â??   
4  Everything Everywhere All at Once (2022)         2022         â??â??Â½   

     Reviewer name Review date  \
0           Tuomas   12-Jan-20   
1             Joao   20-Dec-22   
2         NicoPico   15-Sep-22   
3        Ella Kemp    8-Apr-22   
4  CosmonautMarkie   14-Aug-19   

                                              Review Comment count  \
0                  This review may contain spoilers.           130   
1  if youâ??ve never swam in the ocean then of co...          1.8K   
2                Puss in Boots: Into the Pussy-Verse          6  2   
3  I will NOT leave my donkey outside when Iâ??m sad

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Downloading required NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')

# Defining the preprocessing function
def preprocess_text(text):
    # Check if text is a valid string
    if isinstance(text, str):
        stop_words = set(stopwords.words('english'))  # Get English stopwords
        lemmatizer = WordNetLemmatizer()  # Initialize lemmatizer
        tokens = text.split()  # Split text into words
        # Remove stopwords and lemmatize
        tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens if word.lower() not in stop_words]
        return " ".join(tokens)
    else:
        # If text is not valid, return an empty string
        return ""

# Applying the preprocessing function to the 'Review' column in the first dataset
data['cleaned_text'] = data['Review'].apply(preprocess_text)

# Applying the preprocessing function to the 'summary' column in the second dataset
data2['cleaned_text'] = data2['summary'].apply(preprocess_text)

# To display the first few rows of the cleaned text from both datasets
print("First dataset cleaned text:")
print(data[['Review', 'cleaned_text']].head())

print("\nSecond dataset cleaned text:")
print(data2[['summary', 'cleaned_text']].head())


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\b0161166\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\b0161166\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


First dataset cleaned text:
                                              Review  \
0                  This review may contain spoilers.   
1  if youâ??ve never swam in the ocean then of co...   
2                Puss in Boots: Into the Pussy-Verse   
3  I will NOT leave my donkey outside when Iâ??m sad   
4  Watch it and have fun before film Twitter tell...   

                                        cleaned_text  
0                       review may contain spoilers.  
1  youâ??ve never swam ocean course pool seems deep.  
2                             pus boots: pussy-verse  
3                     leave donkey outside iâ??m sad  
4       watch fun film twitter tell itâ??s overrated  

Second dataset cleaned text:
                                             summary  \
0  This film noir portrait of corruption and mora...   
1  Seven Samurai (Shichinin no samurai) tells the...   
2  An aging group of outlaws look for one last bi...   
3  Robert Bresson's 1966 film focuses on the stor..

In [3]:
import gensim
from gensim import corpora
from gensim.models import LdaModel

# Step 1: Define the function to apply LDA and display topics in descending order of frequency
def apply_lda_and_display_topics(data, num_topics_list, dataset_name="Dataset"):
    # Tokenize the cleaned text
    texts = [text.split() for text in data['cleaned_text']]

    # Create a dictionary and corpus for the dataset
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]

    # Looping through each value of N (number of topics)
    for num_topics in num_topics_list:
        lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)
        
        print(f"\nTopics for {dataset_name} (N={num_topics}):")
        
        # Get the topics sorted by frequency (descending order)
        topic_list = lda_model.print_topics(num_words=5)
        
        # Calculate frequency of each topic
        topic_frequency = {}
        for doc in corpus:
            topics = lda_model.get_document_topics(doc)
            for topic_id, topic_prob in topics:
                if topic_id not in topic_frequency:
                    topic_frequency[topic_id] = topic_prob
                else:
                    topic_frequency[topic_id] += topic_prob

        # Sort topics by frequency in descending order
        sorted_topic_frequency = sorted(topic_frequency.items(), key=lambda x: x[1], reverse=True)
        
        # Display the sorted topics in descending order of frequency
        for topic_id, freq in sorted_topic_frequency:
            print(f"Topic {topic_id+1}: {lda_model.print_topic(topic_id)} - Frequency: {freq}")

# Step 2: Apply LDA to both datasets
num_topics_list = [5, 10, 20, 50]

# Apply LDA and display topics for the first dataset
apply_lda_and_display_topics(data, num_topics_list, dataset_name="Dataset 1")

# Apply LDA and display topics for the second dataset
apply_lda_and_display_topics(data2, num_topics_list, dataset_name="Dataset 2")



Topics for Dataset 1 (N=5):
Topic 3: 0.022*"movie" + 0.019*"like" + 0.008*"one" + 0.007*"film" + 0.005*"would" + 0.005*"feel" + 0.005*"ever" + 0.005*"think" + 0.004*"fucking" + 0.004*"good" - Frequency: 1138.279296875
Topic 5: 0.016*"movie" + 0.007*"like" + 0.007*"want" + 0.006*"make" + 0.006*"get" + 0.006*"time" + 0.006*"itâ??s" + 0.005*"film" + 0.005*"every" + 0.004*"really" - Frequency: 1013.9064331054688
Topic 1: 0.006*"gay" + 0.006*"like" + 0.004*"time" + 0.004*"one" + 0.004*"iâ??m" + 0.004*"get" + 0.004*"girl" + 0.004*"film" + 0.004*"watching" + 0.004*"fucking" - Frequency: 904.466796875
Topic 2: 0.011*"movie" + 0.007*"love" + 0.006*"see" + 0.006*"ever" + 0.005*"like" + 0.005*"people" + 0.005*"know" + 0.005*"one" + 0.004*"man" + 0.004*"film" - Frequency: 899.5299072265625
Topic 4: 0.069*"?â" + 0.027*"â" + 0.004*"go" + 0.003*"good" + 0.003*"cry" + 0.003*"would" + 0.003*"\_" + 0.003*"_/" + 0.003*"movie" + 0.003*"making" - Frequency: 632.9182739257812

Topics for Dataset 1 (N=10):



Topics for Dataset 2 (N=5):
Topic 4: 0.009*"life" + 0.007*"new" + 0.007*"find" + 0.007*"young" + 0.006*"one" + 0.005*"two" + 0.005*"family" + 0.004*"love" + 0.004*"take" + 0.004*"woman" - Frequency: 6936.876953125
Topic 2: 0.010*"film" + 0.008*"story" + 0.004*"one" + 0.004*"life" + 0.004*"documentary" + 0.004*"american" + 0.003*"world" + 0.003*"films)" + 0.003*"war" + 0.003*"first" - Frequency: 2701.322265625
Topic 1: 0.005*"must" + 0.004*"team" + 0.003*"agent" + 0.003*"find" + 0.003*"get" + 0.003*"save" + 0.003*"go" + 0.003*"-" + 0.002*"take" + 0.002*"help" - Frequency: 1673.2886962890625
Topic 5: 0.002*"jack" + 0.002*"henry" + 0.001*"dying" + 0.001*"(kevin" + 0.001*"hunter" + 0.001*"grant" + 0.001*"mary" + 0.001*"vow" + 0.001*"pal" + 0.001*"hole" - Frequency: 776.2637329101562
Topic 3: 0.006*"century" + 0.004*"fox)" + 0.004*"(20th" + 0.003*"cinema)" + 0.003*"based" + 0.003*"(new" + 0.002*"line" + 0.002*"alien" + 0.002*"--" + 0.002*"serial" - Frequency: 742.9730224609375

Topics for 

Topic 37: 0.038*"line" + 0.021*"(new" + 0.020*"sam" + 0.019*"cinema)" + 0.011*"max" + 0.009*"l.a." + 0.008*"black" + 0.008*"inadvertently" + 0.007*"agency" + 0.007*"child." - Frequency: 170.46939086914062
Topic 38: 0.045*"pictures)" + 0.032*"classic" + 0.028*"(paramount" + 0.024*"king" + 0.023*"tale" + 0.022*"(universal" + 0.022*"comic" + 0.017*"queen" + 0.012*"elizabeth" + 0.011*"version" - Frequency: 159.50271606445312
Topic 44: 0.096*"comedy" + 0.056*"romantic" + 0.023*"tom" + 0.021*"dog" + 0.014*"hot" + 0.013*"guy" + 0.010*"wind" + 0.009*"love" + 0.008*"tough" + 0.008*"romance." - Frequency: 146.554931640625
Topic 6: 0.064*"century" + 0.037*"fox)" + 0.032*"(20th" + 0.013*"security" + 0.010*"alive." + 0.009*"video" + 0.008*"revenge," + 0.008*"technology" + 0.008*"everyday" + 0.008*"internet" - Frequency: 145.5489044189453
Topic 18: 0.013*"scientist" + 0.013*"pictures]" + 0.012*"holiday" + 0.011*"protect" + 0.009*"completely" + 0.009*"darkness" + 0.009*"hate" + 0.008*"honor" + 0.008*

In [8]:
import plotly.express as px
import plotly.offline as pyo
import plotly.io as pio
import gensim
from gensim import corpora
from gensim.models import LdaModel

# Initialize Plotly in notebook mode
pyo.init_notebook_mode(connected=True)

# Define the function to apply LDA and plot donut charts
def apply_lda_and_plot_donut(data, num_topics_list, dataset_name="Dataset"):
    # Tokenize the cleaned text
    texts = [text.split() for text in data['cleaned_text']]

    # Create a dictionary and corpus for the dataset
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]

    # Loop through each value of N (number of topics)
    for num_topics in num_topics_list:
        lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

        # Get the topic frequencies
        topic_frequency = {i: 0 for i in range(num_topics)}
        for doc in corpus:
            topics = lda_model.get_document_topics(doc)
            for topic_id, topic_prob in topics:
                topic_frequency[topic_id] += topic_prob
        
        # Prepare data for plotting
        topic_labels = [f"Topic {i+1}" for i in range(num_topics)]
        topic_counts = [topic_frequency[i] for i in range(num_topics)]

        # Plot donut chart for the current dataset and number of topics
        fig = px.pie(names=topic_labels, values=topic_counts, hole=0.3, title=f"Topic Distribution in {dataset_name} (N={num_topics})")
        
        # To ensure plot is displayed using plotly.io.show()
        pio.show(fig)

# Step 2: Apply LDA to both datasets for 5, 10, 20 topics
num_topics_list = [5, 10, 20]

# Applying LDA and plot donut charts for the first dataset
apply_lda_and_plot_donut(data, num_topics_list, dataset_name="Dataset 1")

# Applying LDA and plot donut charts for the second dataset
apply_lda_and_plot_donut(data2, num_topics_list, dataset_name="Dataset 2")


In [9]:
# To save the figures as HTML files with specific paths for each dataset and number of topics
def save_figures_as_html(fig, num_topics, dataset_name, output_dir="C:/Users/b0161166/Downloads/"):
    # Create a unique filename based on the dataset name and number of topics
    output_file = f"{output_dir}{dataset_name}_topic_distribution_N{num_topics}.html"
    fig.write_html(output_file)
    print(f"Saved {output_file}")

# Inside the apply_lda_and_plot_donut function, we shall save each figure as HTML:
def apply_lda_and_plot_donut(data, num_topics_list, dataset_name="Dataset", output_dir="C:/Users/b0161166/Downloads/"):
    # Tokenize the cleaned text
    texts = [text.split() for text in data['cleaned_text']]

    # Create a dictionary and corpus for the dataset
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]

    # Loop through each value of N (number of topics)
    for num_topics in num_topics_list:
        lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

        # Get the topic frequencies
        topic_frequency = {i: 0 for i in range(num_topics)}
        for doc in corpus:
            topics = lda_model.get_document_topics(doc)
            for topic_id, topic_prob in topics:
                topic_frequency[topic_id] += topic_prob

        # Preparing data for plotting
        topic_labels = [f"Topic {i+1}" for i in range(num_topics)]
        topic_counts = [topic_frequency[i] for i in range(num_topics)]

        # To plot donut chart for the current dataset and number of topics
        fig = px.pie(names=topic_labels, values=topic_counts, hole=0.3, title=f"Topic Distribution in {dataset_name} (N={num_topics})")
        
        # To save the figure as an HTML file using the helper function
        save_figures_as_html(fig, num_topics, dataset_name, output_dir)

# Applying LDA to both datasets for 5, 10, 20 topics
num_topics_list = [5, 10, 20]

# Applying LDA and plot donut charts for the first dataset and save HTML files
apply_lda_and_plot_donut(data, num_topics_list, dataset_name="Dataset_1")

# Applying LDA and plot donut charts for the second dataset and save HTML files
apply_lda_and_plot_donut(data2, num_topics_list, dataset_name="Dataset_2")


Saved C:/Users/b0161166/Downloads/Dataset_1_topic_distribution_N5.html
Saved C:/Users/b0161166/Downloads/Dataset_1_topic_distribution_N10.html
Saved C:/Users/b0161166/Downloads/Dataset_1_topic_distribution_N20.html
Saved C:/Users/b0161166/Downloads/Dataset_2_topic_distribution_N5.html
Saved C:/Users/b0161166/Downloads/Dataset_2_topic_distribution_N10.html
Saved C:/Users/b0161166/Downloads/Dataset_2_topic_distribution_N20.html
